In [1]:
from tqdm import tqdm
from typing import List, Dict
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flowprintOptimal.sekigo.core.flowRepresentation import FlowRepresentation,PacketFlowRepressentation,TimeslotRepresentation
from flowprintOptimal.sekigo.dataAnalysis.vNATDataFrameProcessor import VNATDataFrameProcessor
from flowprintOptimal.sekigo.core.flowConfig import FlowConfig
import random
from flowprintOptimal.sekigo.flowUtils.flowDatasets import PacketFlowDataset, BaseFlowDataset, DDQNActivityDataset
from torch.utils.data import Dataset,DataLoader
from torchsampler import ImbalancedDatasetSampler
from sklearn.model_selection import train_test_split
from flowprintOptimal.sekigo.flowUtils.commons import normalizePacketRep
import os
from joblib import Parallel, delayed
from flowprintOptimal.sekigo.flowUtils.commons import saveFlows,loadFlows
from flowprintOptimal.sekigo.dataAnalysis.dataFrameProcessor import UTMobileNetProcessor
from flowprintOptimal.sekigo.flowUtils.dataGetterV2 import getTrainTestOODBalanced, saveTrainTestOODFlows, getFlowReps
from flowprintOptimal.sekigo.flowUtils.conversions import convertPacketRepToTimeslotRepEffecient
from sklearn.metrics import confusion_matrix
import json
from flowprintOptimal.sekigo.modeling.trainers import NNClassificationTrainer
from flowprintOptimal.sekigo.modeling.neuralNetworks import LSTMNetwork,TransformerGenerator,CNNNetwork1D, LSTMDuelingNetwork
from flowprintOptimal.sekigo.modeling.loggers import Logger
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
from flowprintOptimal.sekigo.earlyClassification.DQL.core import MemoryElement,Rewarder,State
from flowprintOptimal.sekigo.earlyClassification.DQL.memoryFiller import MemoryFiller
from flowprintOptimal.sekigo.earlyClassification.DQL.datasets import MemoryDataset
from flowprintOptimal.sekigo.earlyClassification.DQL.trainers import EarlyClassificationtrainer
from flowprintOptimal.sekigo.utils.documentor import Documenter
from flowprintOptimal.sekigo.utils.evaluations import Evaluator,EarlyEvaluation
import warnings
from copy import deepcopy
import torch.nn as nn
warnings.filterwarnings('ignore')

In [ ]:
train_packet_flows,test_packet_flows,ood_packet_flows, train_timeslot_flows,test_timeslot_flows,ood_timeslot_flows = getTrainTestOODBalanced(dataset_name= "deployment", test_size= .2, ood_classes= [],max_flow_length_in_seconds= 240, packet_max_length= 500)

2992
2992
{'File Storage': 856, 'Download': 536, 'Software Update': 411, 'Video': 323, 'Mail': 251, 'Social Media': 160}
keep_number = 856
------------------------------
train class distrubation
File Storage       673
Download           423
Software Update    341
Video              259
Mail               200
Social Media       133
Name: count, dtype: int64
test class distrubation
File Storage       183
Download           113
Software Update     70
Video               64
Mail                51
Social Media        27
Name: count, dtype: int64


In [ ]:
saveTrainTestOODFlows(base_path= "data/ClassificationOnlyFlows", dataset_name= "deployment", 
                      train_packet_flows= train_packet_flows, test_packet_flows= test_packet_flows,
                      train_timeslot_flows= train_timeslot_flows, test_timeslot_flows = test_timeslot_flows,
                      ood_packet_flows= ood_packet_flows, ood_timeslot_flows = ood_timeslot_flows
                      )

In [ ]:
lens = [len(x) for x in train_timeslot_flows]

In [10]:
pd.Series(lens).describe()

count    2029.000000
mean       13.661902
std         2.103588
min         8.000000
25%        13.000000
50%        15.000000
75%        15.000000
max        15.000000
dtype: float64